In [1]:
import itertools
max_degree=4
n=2
gen=list(range(max_degree+4))
gen_lists=itertools.product(gen,repeat=4)
list_of_degree=[[] for _ in range(max_degree+2)]
for gen_list in gen_lists:
    sum_of_degree=sum(gen_list)
    if 2<=sum_of_degree<=max_degree+3:
        list_of_degree[sum(gen_list)-2].append(gen_list)

In [26]:
#特定の次数の項を取り出す
def extract_degree(f,x,degree):
    answer=f
    for num,i in enumerate(degree):
        answer=answer.coefficient(x[num],i)
    try:
        answer=factor(answer.simplify_full())
    except:
        pass
    return answer
#係数にベキ式を追加する
def recover_term(term,x,degree):
    answer=term
    for num,i in enumerate(degree):
        answer*=x[num]^i
    return answer

#Poisson括弧の定義
def poisson(f,g):
    if f==0 or g==0:
        return 0
    else:
        poisson_answer=0
        for i in range(n):
            poisson_answer+=f.diff(x[i])*g.diff(x[i+n])-g.diff(x[i])*f.diff(x[i+n])
        return poisson_answer
#コンビネーション
#nCkの計算
def combination(n,k):
    return factorial(n)/factorial(k)/factorial(n-k)

#h[i][j]の更新式
def lie_element(i,j):
    answer=h[i-1][j+1]
    for k in range(j+1):
        answer+=combination(j,k)*poisson(h[i-1][j-k],w[k+1])

#
def EqHofN(n,h,w):
    tmp=0
    for i in range(1,n):
        for k in range(n-i+1):
            tmp+=poisson(h[k][n-i-k],w[i])
    return tmp+h[0][n]

In [19]:
var('x1,x2,y1,y2,p1,p2,q1,q2')
var('alpha,omega,delta')
var('p_r,p_z,r,z')
var('a,b,c')
x=[x1,x2,y1,y2]

In [20]:
a=omega^(3/2)*alpha/(4*alpha+1)
b=omega^2*alpha/(4*alpha+1)
c=omega^(3/2)*alpha^(1/2)*(alpha+2)^(1/4)/(sqrt(2)*(4*alpha+1)^(3/4))

ハミルトニアンの定義

In [21]:
H=p_r*p_r/4+omega*omega/(4*r*r)+(alpha+2)*p_z*p_z/(4*alpha)-1/(2*alpha*r)-2/sqrt(r*r+z*z);show(H)

1/4*p_r^2 + 1/4*(alpha + 2)*p_z^2/alpha + 1/4*omega^2/r^2 - 2/sqrt(r^2 + z^2) - 1/2/(alpha*r)

線形変換してテイラー展開

In [22]:
H=H.substitute(r=a*q1+b,p_r=p1/a,z=c*q2,p_z=p2/c)

H=taylor(H,(q1,0),(q2,0),(p1,0),(p2,0),max_degree)

H=simplify(expand(H.substitute(p1=(x1-I*y1)/sqrt(2),q1=(-I*x1+y1)/sqrt(2),p2=(x2-I*y2)/sqrt(2),q2=(-I*x2+y2)/sqrt(2))))

ハミルトニアンを次数ごとに分解

In [30]:
h=[[0 for _ in range(max_degree)] for _ in range(max_degree)]
sum_of_degree=0
for list_of_degree_n in list_of_degree[:max_degree-1]:
    #n次の次数のハミルトニアンを考える
    hamiltonian_of_degree_n=0
    for degree in list_of_degree_n:
        tmp=extract_degree(H,x,degree)
        for num,i in enumerate(degree):
            tmp*=x[num]^i
        hamiltonian_of_degree_n+=tmp
    h[0][sum_of_degree]=hamiltonian_of_degree_n
    sum_of_degree+=1

これで6次までの展開が出来た。
次にLie Transformationを行う。

In [31]:
w=[0 for _ in range(max_degree)]
for i in range(1,max_degree):
    #はじめにwを求める
    for s in list_of_degree[i]:
        s1,s2,t1,t2=s
        if s[:n]!=s[n:]:
            u=var('u')
            w_tmp=recover_term(u,x,s)
            #ここでwを求める
            #ここはi=1の更新式だけ
            eqn=expand(EqHofN(i,h,w)+poisson(h[0][0],w_tmp))
            t=solve(extract_degree(eqn,x,s),u)[0].rhs()
            try:
                t=factor(t)
            except:
                pass
            w_tmp=(w_tmp)(u=t)
            w[i]+=w_tmp
    #次にhの要素を求める

In [32]:
show(w)

[0,
 1/12*sqrt(2)*x1^3/sqrt(omega) + 3/4*sqrt(2)*sqrt(4*alpha + 1)*sqrt(alpha + 2)*x1*x2^2/((4*sqrt(4*alpha + 1)*sqrt(alpha + 2) + 4*alpha + 1)*sqrt(omega)) + 3/4*I*sqrt(2)*x1^2*y1/sqrt(omega) + 3/4*I*sqrt(2)*sqrt(4*alpha + 1)*sqrt(alpha + 2)*x2^2*y1/((4*sqrt(4*alpha + 1)*sqrt(alpha + 2) - 4*alpha - 1)*sqrt(omega)) + 3/4*sqrt(2)*x1*y1^2/sqrt(omega) + 1/12*I*sqrt(2)*y1^3/sqrt(omega) + 3/2*I*sqrt(2)*sqrt(alpha + 2)*x1*x2*y2/(sqrt(4*alpha + 1)*sqrt(omega)) + 3/2*sqrt(2)*sqrt(alpha + 2)*x2*y1*y2/(sqrt(4*alpha + 1)*sqrt(omega)) + 3/4*sqrt(2)*sqrt(4*alpha + 1)*sqrt(alpha + 2)*x1*y2^2/((4*sqrt(4*alpha + 1)*sqrt(alpha + 2) - 4*alpha - 1)*sqrt(omega)) + 3/4*I*sqrt(2)*sqrt(4*alpha + 1)*sqrt(alpha + 2)*y1*y2^2/((4*sqrt(4*alpha + 1)*sqrt(alpha + 2) + 4*alpha + 1)*sqrt(omega)),
 9/32*I*x1^4/omega + (1936*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 + 3904*alpha^3 + 6568*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha + 17824*alpha^2 + 5441*sqrt(4*alpha + 1)*sqrt(alpha + 2) + 25428*alpha + 10792)*(9*I*alpha + 18*I)*x1^2*x2^2/((2592*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 + 5184*alpha^3 + 6224*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha + 18224*alpha^2 + 2962*sqrt(4*alpha + 1)*sqrt(alpha + 2) + 17548*alpha + 3329)*(12*alpha + 31)*omega) + 3/256*I*(12*alpha - 31)*sqrt(4*alpha + 1)*sqrt(alpha + 2)*x2^4/((12*alpha + 31)*alpha*omega) + 3/4*x1^3*y1/omega + 3/2*(6*alpha + 19)*x1*x2^2*y1/((12*alpha + 31)*omega) - (1936*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 - 3904*alpha^3 + 6568*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha - 17824*alpha^2 + 5441*sqrt(4*alpha + 1)*sqrt(alpha + 2) - 25428*alpha - 10792)*(9*I*alpha + 18*I)*x2^2*y1^2/((2592*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 - 5184*alpha^3 + 6224*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha - 18224*alpha^2 + 2962*sqrt(4*alpha + 1)*sqrt(alpha + 2) - 17548*alpha - 3329)*(12*alpha + 31)*omega) + 3/4*x1*y1^3/omega - 9/32*I*y1^4/omega + 18*(alpha + 2)^(3/2)*x1^2*x2*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*omega) + 18*(alpha + 2)^(3/2)*x2*y1^2*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*omega) + (1936*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 - 3904*alpha^3 + 6568*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha - 17824*alpha^2 + 5441*sqrt(4*alpha + 1)*sqrt(alpha + 2) - 25428*alpha - 10792)*(9*I*alpha + 18*I)*x1^2*y2^2/((2592*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 - 5184*alpha^3 + 6224*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha - 18224*alpha^2 + 2962*sqrt(4*alpha + 1)*sqrt(alpha + 2) - 17548*alpha - 3329)*(12*alpha + 31)*omega) + 3/2*(6*alpha + 19)*x1*y1*y2^2/((12*alpha + 31)*omega) - (1936*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 + 3904*alpha^3 + 6568*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha + 17824*alpha^2 + 5441*sqrt(4*alpha + 1)*sqrt(alpha + 2) + 25428*alpha + 10792)*(9*I*alpha + 18*I)*y1^2*y2^2/((2592*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha^2 + 5184*alpha^3 + 6224*sqrt(4*alpha + 1)*sqrt(alpha + 2)*alpha + 18224*alpha^2 + 2962*sqrt(4*alpha + 1)*sqrt(alpha + 2) + 17548*alpha + 3329)*(12*alpha + 31)*omega) - 3/256*I*(12*alpha - 31)*sqrt(4*alpha + 1)*sqrt(alpha + 2)*y2^4/((12*alpha + 31)*alpha*omega) + 3/32*(144*alpha^2 + 496*alpha + 31)*sqrt(alpha + 2)*x2^3*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*alpha*omega) + 3/32*(144*alpha^2 + 496*alpha + 31)*sqrt(alpha + 2)*x2*y2^3/((12*alpha + 31)*sqrt(4*alpha + 1)*alpha*omega),
 21/160*sqrt(2)*x1^5/omega^(3/2) + 9/8*I*sqrt(2)*x1^4*y1/omega^(3/2) - 219/32*sqrt(2)*x1^3*y1^2/omega^(3/2) + 219/32*I*sqrt(2)*x1^2*y1^3/omega^(3/2) - 9/8*sqrt(2)*x1*y1^4/omega^(3/2) - 21/160*I*sqrt(2)*y1^5/omega^(3/2) - 1/112*I*sqrt(2)*(576*alpha^2 + 36*alpha - 3359)*sqrt(alpha + 2)*x1^3*x2*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*omega^(3/2)) + 3/56*sqrt(2)*(288*alpha^2 - 3972*alpha - 11203)*sqrt(alpha + 2)*x1^2*x2*y1*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*omega^(3/2)) - 3/56*I*sqrt(2)*(288*alpha^2 - 3972*alpha - 11203)*sqrt(alpha + 2)*x1*x2*y1^2*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*omega^(3/2)) + 1/112*sqrt(2)*(576*alpha^2 + 36*alpha - 3359)*sqrt(alpha + 2)*x2*y1^3*y2/((12*alpha + 31)*sqrt(4*alpha + 1)*omega^(3/2)) + 21/32*s

In [11]:
show(h)

[[-1/2*I*(4*alpha + 1)^2*x1*y1/(alpha^2*omega^3) - I*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x2*y2/(alpha^2*omega^3),
  -1/8*I*sqrt(2)*(4*alpha + 1)^2*x1^3/(alpha^2*omega^(7/2)) - 3/8*I*sqrt(2)*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1*x2^2/(alpha^2*omega^(7/2)) + 3/8*sqrt(2)*(4*alpha + 1)^2*x1^2*y1/(alpha^2*omega^(7/2)) + 3/8*sqrt(2)*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x2^2*y1/(alpha^2*omega^(7/2)) + 3/8*I*sqrt(2)*(4*alpha + 1)^2*x1*y1^2/(alpha^2*omega^(7/2)) - 1/8*sqrt(2)*(4*alpha + 1)^2*y1^3/(alpha^2*omega^(7/2)) + 3/4*sqrt(2)*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1*x2*y2/(alpha^2*omega^(7/2)) + 3/4*I*sqrt(2)*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x2*y1*y2/(alpha^2*omega^(7/2)) + 3/8*I*sqrt(2)*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1*y2^2/(alpha^2*omega^(7/2)) - 3/8*sqrt(2)*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*y1*y2^2/(alpha^2*omega^(7/2)),
  3/16*(4*alpha + 1)^2*x1^4/(alpha^2*omega^4) + 3/4*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1^2*x2^2/(alpha^2*omega^4) - 3/64*(4*alpha + 1)^2*(alpha + 2)*x2^4/(alpha^3*omega^4) + 3/4*I*(4*alpha + 1)^2*x1^3*y1/(alpha^2*omega^4) + 3/2*I*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1*x2^2*y1/(alpha^2*omega^4) - 9/8*(4*alpha + 1)^2*x1^2*y1^2/(alpha^2*omega^4) - 3/4*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x2^2*y1^2/(alpha^2*omega^4) - 3/4*I*(4*alpha + 1)^2*x1*y1^3/(alpha^2*omega^4) + 3/16*(4*alpha + 1)^2*y1^4/(alpha^2*omega^4) + 3/2*I*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1^2*x2*y2/(alpha^2*omega^4) - 1/16*(4*alpha + 1)^2*(3*I*alpha + 6*I)*x2^3*y2/(alpha^3*omega^4) - 3*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1*x2*y1*y2/(alpha^2*omega^4) - 3/2*I*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x2*y1^2*y2/(alpha^2*omega^4) - 3/4*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1^2*y2^2/(alpha^2*omega^4) + 9/32*(4*alpha + 1)^2*(alpha + 2)*x2^2*y2^2/(alpha^3*omega^4) - 3/2*I*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*x1*y1*y2^2/(alpha^2*omega^4) + 3/4*(4*alpha + 1)^(3/2)*sqrt(alpha + 2)*y1^2*y2^2/(alpha^2*omega^4) + 1/16*(4*alpha + 1)^2*(3*I*alpha + 6*I)*x2*y2^3/(alpha^3*omega^4) - 3/64*(4*alpha + 1)^2*(alpha + 2)*y2^4/(alpha^3*omega^4)],
 [0, 0, 0],
 [0, 0, 0]]